In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('/app')

In [3]:

# %%
from qdrant_client import QdrantClient
from datetime import datetime, timedelta
import pandas as pd

In [6]:

pd.set_option('display.max_colwidth', None)

# %% [markdown]
# ## Connect to Qdrant

# %%
def get_qdrant_client():
    """Get Qdrant client with default settings"""
    return QdrantClient(
        host="qdrant",  # Update if running in different environment
        port=6333
    )

# %% [markdown]
# ## Query Recent Chunks

# %%
def get_recent_chunks(collection_name="meeting_chunks", limit=100, hours_ago=24000*30):
    """
    Retrieve the most recent chunks from Qdrant.
    
    Args:
        collection_name (str): Name of the Qdrant collection
        limit (int): Maximum number of chunks to retrieve
        hours_ago (int): Look back period in hours
    
    Returns:
        pd.DataFrame: DataFrame containing chunk data
    """
    client = get_qdrant_client()
    
    # Calculate timestamp threshold
    time_threshold = (datetime.utcnow() - timedelta(hours=hours_ago)).isoformat()
    
    # Search with filtering by timestamp
    search_result = client.scroll(
        collection_name=collection_name,
        scroll_filter={"must": [
            {"key": "timestamp", "range": {"gte": time_threshold}}
        ]},
        limit=limit,
        with_payload=True,
        with_vectors=True
    )[0]  # scroll returns (points, next_page_offset)
    
    # Extract relevant fields from payload
    chunks_data = []
    for point in search_result:
        payload = point.payload
        chunks_data.append({
            'id': point.id,
            'timestamp': pd.to_datetime(payload.get('timestamp')),  # Convert to datetime
            'content_id': payload.get('content_id'),
            'content_type': payload.get('content_type'),
            'content': payload.get('content'),
            'contextualized_content': payload.get('contextualized_content'),
            'chunk_index': payload.get('chunk_index'),
            'topic': payload.get('topic'),
            'speaker': payload.get('speaker'),
            'speakers': payload.get('speakers'),
            'vector_dim': len(point.vector) if point.vector else None
        })
    
    # Convert to DataFrame
    df = pd.DataFrame(chunks_data)
    

    return df

In [7]:
get_recent_chunks()

,id,timestamp,content_id,content_type,content,contextualized_content,chunk_index,topic,speaker,speakers,vector_dim
0,04985fba-0412-4b0e-8a8d-1d69ae484286,2025-01-23 09:40:30.720000+00:00,580bb613-00de-4c4a-b048-2c2576fcdef0,meeting,": Я думаю, да, сможешь. Ладно, ребят, я должен закончить сейчас. Пора кушать, да. У меня пришел человек. Хорошо. Пока-пока.","The chunk occurs towards the end of a conversation between Artem Puzik and Dmitry Grankin, where they discuss project details and technical aspects. Dmitry indicates he needs to leave for a meal, signaling the conclusion of their discussion.",16,Meeting Conclusion,,"[Artem Puzik, Sergey Ryabenko, Dmitry Grankin]",1024
1,05cdbf79-f791-489a-bdbf-40084402e8f9,2025-01-15 14:07:51.364000+00:00,fb996504-0475-4ee9-a85e-dda4e9f15a4b,meeting,"Alex Shevliakov: тогда можно вот здесь сделать переключение между чат ноут и ноут редактор сам текста будет заменять сам чат то есть каждый Это раз, когда ты сохраняешь ноут и переходишь в режим ноута, он будет менять вот этот интерфейс на текстовый редактор. Но это зависит скорее от... мне кажется что лучше иметь возможность параллельно видеть чай чат ноут мне кажется что логично когда если но вот это тоже просто там у нас транскрипт, а здесь у нас редактируемый транскрипт. Та же самая история. Еще мысли по поводу того ноут это отдельная колонка или внутри вместе с звонками а если у нас будет множество разных Если у нас будут звонки, может быть, YouTube-видео, может быть, документы, заметки, букмарки. Всякие, да. Понял. То есть необязательно, что это... Необязательно называть вкладку Calls.","The chunk is part of a discussion between Alex Shevliakov and Dmitriy Grankin about the user interface design for a chat and note-taking application. They are exploring the functionality of switching between a chat interface and a note editor, considering how to organize various content types such as calls, YouTube videos, documents, and bookmarks within the application.",27,Note Editing,Alex Shevliakov,"[Alex Shevliakov, Dmitriy Grankin]",1024
2,06798dd2-0563-46b0-9256-08092fe30c06,2025-01-21 15:02:50.184000+00:00,a0c3f703-2a5e-4ed0-9561-807121516984,meeting,"Dmitry Grankin: Am I right saying that Lisbon Unicorn Factory is established with the help of Lisbon City Council, right? Yes, you are right. so that you've got facilities which you basically want to contribute to the development of AI startups.","The chunk occurs during a conversation between Mariana Montenegro and Dmitry Grankin, where they discuss the establishment and purpose of the Lisbon Unicorn Factory, specifically focusing on its role in supporting AI startups and its collaboration with the Lisbon City Council.",4,Support from Lisbon City Council,Dmitry Grankin,"[Mariana Montenegro, Dmitry Grankin, Pedro Teixeira]",1024
3,15cec577-9b5f-4ca4-9bfb-21667b7f5a10,2025-01-15 14:07:51.364000+00:00,fb996504-0475-4ee9-a85e-dda4e9f15a4b,meeting,"Alex Shevliakov: general search и этот фильтр суши выглядит так что как будто бы юзер когда приходит у него самое главное это интерфейс чата и фильтр которые на самом деле плоские, то есть я бы выделил просто в одну область эти все фильтры и сделал между ними разграничение, что ты можешь выбрать такой, такой и или такой фильтр. А звонки? Звонки также. То есть, например, давай визуально переключимся. Я бы, наверное, сделал, что возьмем старый фрейм. Без контента берем вот это. Вообще, уберем вот это пока как не нужно. Это будет в центре. И уберем, наверное, даже вот так пока. Это был интерфейс чата. Вот он. То есть совсем грубо говоря... Ой, не сюда попал. Mm-hmm. Здесь, например, были бы «speakers», нет, не так, «calls». Curse и Text, можно еще какие-то потом пока добавить.","This chunk is part of a discussion between Alex Shevliakov and Dmitriy Grankin about the user interface design for a chat application. Alex is suggesting improvements to the layout, specifically focusing on how filters and calls should be organized within the chat interface to enhance use